In [44]:
# Импортируем библиотеки, которые скорее всего понядобятся.

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm.notebook import tqdm
import re

In [63]:
# Считаем то, что получилось, из файла.

df = pd.read_csv('parsed_chitay_gorod_kids_02.csv')
print(df.shape)
df.tail()

(3384, 31)


,ID товара,Название,Автор,Цикл,Том,Категория,Подкатегории,Издательство,Серия,Год издания,...,Кол-во страниц,Тип обложки,Формат,Вес,Тираж,Наличие на сайте,Поступит в продажу,Наличие в магазинах,Наличие в местных магазинах,Карта
3379,2803206,"Умные загадки, веселые скороговорки и любимые ...",Скиба Т. (сост.),-,-,Детская художественная литература,Стихи и песни,Владис,-,2020,...,64,Твердая глянцевая,24.5 x 17.5 x 0.6,240,5000,Осталось мало,-,40 магазинов,5 магазинов (Москва),https://www.chitai-gorod.ru/shops/reserve/?ele...
3380,2803208,Лучшая книга для маленького эрудита. Что? Когд...,"Феданова Ю., Скиба Т., Машир Т. (ред.)",-,-,Детская научно-популярная литература,Универсальные детские энциклопедии и справочники,Владис,-,2020,...,64,Твердая глянцевая,24.5 x 17.5 x 0.6,240,4000,Осталось мало,-,23 магазина,7 магазинов (Москва),https://www.chitai-gorod.ru/shops/reserve/?ele...
3381,2803209,Россия. Почемучкам и любознашкам,"Феданова Ю., Скиба Т., Машир Т. (ред.)",-,-,Детская научно-популярная литература,Универсальные детские энциклопедии и справочники,Владис,123 вопроса – 123 ответа,2020,...,64,Твердая глянцевая,24.5 x 17.5 x 0.6,240,5000,Осталось мало,-,33 магазина,8 магазинов (Москва),https://www.chitai-gorod.ru/shops/reserve/?ele...
3382,2803840,Секрет лисенка,Бен Фогл,Мистер Пёс,4,Детская художественная литература,"Проза для детей. Повести, рассказы",Эксмо,Мистер Пес спешит на помощь,2020,...,160,Твердая бумажная,20.5 x 13 x 1.2,240,4000,В наличии,-,136 магазинов,20 магазинов (Москва),https://www.chitai-gorod.ru/shops/reserve/?ele...
3383,2803841,Кошачий секрет,"Фрауке Шойнеманн, Антье Циллат",Пауль и Сникерс,2,Детская художественная литература,Приключения,Эксмо,"Пауль и Сникерс, детективы на четырех лапах",2020,...,208,Твердая бумажная,21 x 15 x 1.5,390,4000,В наличии,-,429 магазинов,65 магазинов (Москва),https://www.chitai-gorod.ru/shops/reserve/?ele...


In [62]:
# Посмотрим, как выглядит какая-нибудь строка целиком.

df.loc[3167]

ID товара                                                                2807665
Название                                        Второе криминальное дело Сосыски
Автор                                                              Антон Березин
Цикл                                                                           -
Том                                                                            -
Категория                                      Детская художественная литература
Подкатегории                                                         Приключения
Издательство                                                               Малыш
Серия                                                  СОбачье СЫСКкное Агенство
Год издания                                                                 2020
Возрастные ограничения                                                        6+
Переводчик                                                                     -
Художник                    

In [19]:
# Выведем полностью аннотацию.

df.loc[324]['Аннотация']

'Однажды мама дарит Селесте подарок. И это вовсе не желанный телескоп, а… очки! Селесте непросто к ним сразу привыкнуть — очки давят на уши, запотевают от горячего чая, да еще и одноклассник придумал ей обидное прозвище. Да зачем вообще нужны эти очки?! Для того, чтобы увидеть этот мир во всей полноте его красок, наслаждаться и восхищаться им, а значит, и жить более полной жизнью и становиться благодаря этому лучше! Эта книга поможет всем детям, которым прописали очки, преодолеть сомнения и понять, что очки – это вовсе не проблема, а очень крутая вещь, которая наделяет тебя суперсилой – острым зрением! А специальный разворот с советами от экспертов-офтальмологов о том, как замотивировать ребенка и помочь ему привыкнуть к очкам будет, полезен родителям. Для детей от 4 до 8 лет и их родителей.'

## Доделки и ошибки

**Чтобы дальше работать с этим датасетом, надо провести дебаг, собрать все пропущенные книги, а также
оставшиеся издания детской литературы в разные файлы, а потом склеить всё в один DataFrame.**

- удалить \r из аннотации
- не делать том числом
- не делать кол-во страниц числом
- исправить ISBN (https://www.chitai-gorod.ru/catalog/book/2872501/)
- лучше выводить последний book_error и отмечать страницу, а весь список перезаписывать в файл

In [41]:
book_error = ['https://www.chitai-gorod.ru/catalog/book/2911881/', 'https://www.chitai-gorod.ru/catalog/book/2911233/', 'https://www.chitai-gorod.ru/catalog/book/2907392/', 'https://www.chitai-gorod.ru/catalog/book/2879302/', 'https://www.chitai-gorod.ru/catalog/book/2879301/', 'https://www.chitai-gorod.ru/catalog/book/2879300/', 'https://www.chitai-gorod.ru/catalog/book/2875131/', 'https://www.chitai-gorod.ru/catalog/book/2874478/', 'https://www.chitai-gorod.ru/catalog/book/2872501/', 'https://www.chitai-gorod.ru/catalog/book/2860709/', 'https://www.chitai-gorod.ru/catalog/book/2856315/', 'https://www.chitai-gorod.ru/catalog/book/2846846/', 'https://www.chitai-gorod.ru/catalog/book/2841112/']

for book in book_error:
    with open('book_error.text', 'a') as log:
        log.write(book + '\n')

with open('book_error.text', 'r') as log:
    print(log.read())

https://www.chitai-gorod.ru/catalog/book/2911881/
https://www.chitai-gorod.ru/catalog/book/2911233/
https://www.chitai-gorod.ru/catalog/book/2907392/
https://www.chitai-gorod.ru/catalog/book/2879302/
https://www.chitai-gorod.ru/catalog/book/2879301/
https://www.chitai-gorod.ru/catalog/book/2879300/
https://www.chitai-gorod.ru/catalog/book/2875131/
https://www.chitai-gorod.ru/catalog/book/2874478/
https://www.chitai-gorod.ru/catalog/book/2872501/
https://www.chitai-gorod.ru/catalog/book/2860709/
https://www.chitai-gorod.ru/catalog/book/2856315/
https://www.chitai-gorod.ru/catalog/book/2846846/
https://www.chitai-gorod.ru/catalog/book/2841112/



In [42]:
with open('book_error.text', 'r') as log:
    missed = list(log)
    
print(missed)

['https://www.chitai-gorod.ru/catalog/book/2911881/\n', 'https://www.chitai-gorod.ru/catalog/book/2911233/\n', 'https://www.chitai-gorod.ru/catalog/book/2907392/\n', 'https://www.chitai-gorod.ru/catalog/book/2879302/\n', 'https://www.chitai-gorod.ru/catalog/book/2879301/\n', 'https://www.chitai-gorod.ru/catalog/book/2879300/\n', 'https://www.chitai-gorod.ru/catalog/book/2875131/\n', 'https://www.chitai-gorod.ru/catalog/book/2874478/\n', 'https://www.chitai-gorod.ru/catalog/book/2872501/\n', 'https://www.chitai-gorod.ru/catalog/book/2860709/\n', 'https://www.chitai-gorod.ru/catalog/book/2856315/\n', 'https://www.chitai-gorod.ru/catalog/book/2846846/\n', 'https://www.chitai-gorod.ru/catalog/book/2841112/\n']


In [32]:
with open('book_error.text', 'a') as log:
    log.write(book_error[-1] + '\n')

with open('book_error.text', 'r') as log:
    print(log.read())

https://www.chitai-gorod.ru/catalog/book/2841112/




In [40]:
with open('book_error.text', 'w') as log:
    log.flush()

In [39]:
book_error = ['https://www.chitai-gorod.ru/catalog/book/2911881/', 'https://www.chitai-gorod.ru/catalog/book/2911233/', 'https://www.chitai-gorod.ru/catalog/book/2907392/', 'https://www.chitai-gorod.ru/catalog/book/2879302/', 'https://www.chitai-gorod.ru/catalog/book/2879301/', 'https://www.chitai-gorod.ru/catalog/book/2879300/', 'https://www.chitai-gorod.ru/catalog/book/2875131/', 'https://www.chitai-gorod.ru/catalog/book/2874478/', 'https://www.chitai-gorod.ru/catalog/book/2872501/', 'https://www.chitai-gorod.ru/catalog/book/2860709/', 'https://www.chitai-gorod.ru/catalog/book/2856315/', 'https://www.chitai-gorod.ru/catalog/book/2846846/', 'https://www.chitai-gorod.ru/catalog/book/2841112/']

p = 5

print(str(p) + ' ' + book_error[-1])

5 https://www.chitai-gorod.ru/catalog/book/2841112/


In [45]:
a = pd.DataFrame() # создали пустой датафрейм с помощью метода DataFrame() библиотеки pandas (pd)
a['a'] = [10,20,30] # создаем колонку "а" и помещаем в нее столбец с данными - [10, 20, 30]
a

,a
0,10
1,20
2,30


In [46]:
b = pd.DataFrame() # создали пустой датафрейм с помощью метода DataFrame() библиотеки pandas (pd)
b['a'] = [30,40,50] # создаем колонку "а" и помещаем в нее столбец с данными - [10, 20, 30]
b

,a
0,30
1,40
2,50


In [57]:
c = a.append(b, ignore_index=True)
c.drop_duplicates(inplace=True)
d = c.reset_index(drop=True)
d

C:\Users\Klim\AppData\Local\Temp\ipykernel_5540\3027864119.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  c = a.append(b, ignore_index=True)


,a
0,10
1,20
2,30
3,40
4,50
